In [114]:
import pandas as pd

In [115]:
tipo_peso_cubado = {
    "Mala P": 8,
    "Mala M": 15,
    "Mala G": 20,
    "Mochilete": 8,
    "Sacola com roda": 8,
    "Pasta com roda": 8,
}

In [116]:
itens = pd.read_excel(f"./dados/itens.xlsx")

In [117]:
tipos_reais = []
pesos_cubados = []
for dg in itens[["descricao_item", "GRADE", "peso_liquido"]].values:
    tipo_real = ""
    for tipo in tipo_peso_cubado.keys():
        if "MALA" in tipo.upper():
            if "MALA" in str(dg[0]).upper() and tipo.split(" ")[1].upper() in str(dg[1]).upper():
                tipo_real = tipo
        else:
            if tipo.upper() in str(dg[0]).upper():
                tipo_real = tipo

    if tipo_real == "":
        tipos_reais.append("Diversos")
        pesos_cubados.append(dg[2])
    else:
        tipos_reais.append(tipo_real)
        pesos_cubados.append(tipo_peso_cubado[tipo_real])

itens["tipos_reais"] = tipos_reais
itens["pesos_cubados"] = pesos_cubados
itens["pesos_cubados_real"] = itens["pesos_cubados"]*itens["volumes"]

itens.dropna(subset="serie_nf", inplace=True)
itens["descricao_item"] = ">" + itens["volumes"].astype("string") + " " + itens["GRADE"] + "< " + itens["descricao_item"]
itens["descricao_item"] = itens["descricao_item"].str.replace("  ", "")
itens["serie_nf"] = itens["serie_nf"].astype("int64")

In [119]:
_itens = []
_itens.append(itens.groupby(["nf_numero", "serie_nf"])["descricao_item"].apply(lambda x: " || ".join(map(str, x))))
_itens.append(itens.groupby(["nf_numero", "serie_nf"])[["valor_total", "chave_nfe", "filial", 
    "CNPJ", "pedido", "valor_total_itens", "frete", "largura", "altura", "COMPRIMENTO"]].first())
_itens.append(itens.groupby(["nf_numero", "serie_nf"])[["volumes", "pesos_cubados_real"]].sum())
_itens = pd.concat(_itens, axis=1).sort_values("volumes")
_itens.to_excel("./dados/group_itens.xlsx")